## Stress calculation and fracturing

Below are the functions you created in previous parts.

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pandas as pd
import random
import json

e = 0.03 #relaxation threshold
k = 1.0 #spring constant
l = 1.5 #equilibrium distance this is initially 2 in the lattice but is set to 1 for the simulation
n = 100 # size of the array
fover = 1.1 #over relaxation scale

lattice = np.zeros([n,n,2,1]) #array with particle positions [0] = x, [1] = y
adjacent = np.zeros([n,n]).tolist() #array with list of neighbour particles
stress = np.zeros([n,n,3,1]) #sxx[0], syy[1], sxy[2] stress per particle 


In [ ]:
def triangular():
    global n, lattice
    for i in range(n):
        for j in range(n):
            if (i%2==0 and j%2==0) or (i%2==1 and j%2==1):
                lattice[i][j][0] = i*math.sqrt(3)+2
                lattice[i][j][1] = j+2

In [ ]:
def circle():
    global n, lattice
    for i in range(n):
        for j in range(n):
            length = math.sqrt((i-49)**2 + (j-49)**2)
            if length <= 10:
                lattice[i][j][0] = 0
                lattice[i][j][1] = 0

In [ ]:
def adjacent_particles():
    global lattice, n, adjacent
    neighbours = [(-1,-1),(-1,1),(0,-2),(0,2),(1,-1),(1,1)]
    for i in range(2,n-2):
        for j in range(2,n-2):
            if lattice[i][j].all() != 0:
                adjacent[i][j] = []
                for nb in neighbours:
                    if (i+nb[0]>-1 and i+nb[0]<n) and (j+nb[1]>-1 and j+nb[1]<n):
                        if lattice[i+nb[0]][j+nb[1]].all() !=0:
                            adjacent[i][j].append((i+nb[0],j+nb[1]))


In [ ]:
def force(i, j): 
    global n, k, l, e, fover, stress, lattice #this will allow you to access and modify the variables declared outside this function
    xx, yy, sxx, syy, sxy, anc = 0, 0, 0, 0, 0, 0 
    
# distance between 2 particles 

    for nb in adjacent[i][j]:
        dx = lattice[i][j][0]-lattice[nb[0]][nb[1]][0]
        dy = lattice[i][j][1]-lattice[nb[0]][nb[1]][1]
        dd = math.sqrt(dx**2 + dy**2)


        if (dd != 0):
            uxi = dx/dd
            uyi = dy/dd

        else:  
            uxi = 0.0
            uyi = 0.0

        fn = -k*(dd-l)
        fx = fn*uxi
        fy = fn*uyi
        
# overall force and stress
        xx = xx + fx
        yy = yy + fy
        sxx = sxx + fx*(l/2)*uxi
        syy = syy + fy*(l/2)*uyi
        sxy = sxy + fx*(l/2)*uyi
        anc = anc + k

    stress[i][j][0] = sxx
    stress[i][j][1] = syy
    stress[i][j][2] = sxy
    
# over relaxation step       
    
    anc = 1.0/anc
    xx = xx * anc        
    yy = yy * anc
        
    if (e < math.sqrt((xx*xx)+(yy*yy))): 
        lattice[i][j][0] = lattice[i][j][0] + xx * fover   
        lattice[i][j][1] = lattice[i][j][1] + yy * fover  
       

In [ ]:
def run_simulation():
    global lattice, n
    for z in range(10): #number of time steps
        for i in range(2,n-2): # leaving the fixed borders out
            for j in range(2,n-2):
                if lattice[i][j].all()!=0:
                    force(i, j)

In [ ]:
def stress_calculation():
    global stress, diff_stress, mean_stress
    for i in range(2,n-2): 
        for j in range(2,n-2):
            if lattice[i][j].all()!=0:
                smax = ((stress[i][j][0] + stress[i][j][1]) / 2.0) + math.sqrt(((stress[i][j][0] - stress[i][j][1]) / 2.0) * ((stress[i][j][0] - stress[i][j][1]) / 2.0) + stress[i][j][2] * stress[i][j][2])
                smin = ((stress[i][j][0] + stress[i][j][1]) / 2.0) - math.sqrt(((stress[i][j][0] - stress[i][j][1]) / 2.0) * ((stress[i][j][0] - stress[i][j][1]) / 2.0) + stress[i][j][2] * stress[i][j][2])
                diff_stress[i][j] = smax - smin
                mean_stress[i][j] = (smax + smin)/2

### Part 3 - Fracturing 

Your task is to create a fracturing function that will select one spring from the lattice to break in each loop. The function should calculate fn the same way as the force function. It should then check if fn is bigger or equal to 0. If yes, it calculates ten_break = (fn/tb) + var_i. Var_i is a random number between -0.8 and 0.8 generated using random.uniform(-0.8 , 0.8). To pass the test use the random numbers from the given array called var. Once you do, you can replace var_i with the random.uniform function to create different results.<br>
After calculating ten_break, check if it is bigger than 1, if it is, also compare it to mbreak. If it is greater than mbreak, save the 2 particles: (i,j) and nb as the position where the spring should break. Every time you discover a ten_break that is greater than the current mbreak choose a new spring_to_break along with spring_x and spring_y (i and j). 

In [ ]:
lattice = np.zeros([n,n,2,1])
stress = np.zeros([n,n,3,1])
triangular()
adjacent_particles()
var = np.loadtxt("tests/variation.txt").reshape(50,100,100)
var_i = 0 

In [ ]:
#Task 1
def fracturing(i,j):
    global n, k, l, mbreak, spring_to_break, spring_x, spring_y, var_i
    tb = 1
    ten_break = 0


Each loop (50 steps) should first do over relaxation (2 steps) and then loop over the lattice array to find the spring to break. Once the fracturing finishes, delete the spring from the adjacent array. Remember that each of the positions in the adjacent array has a list of values like this: [(11, 11), (11, 13), (12, 10), (12, 14), (13, 11), (13, 13)]. <br>
If spring_to_break = (11, 11) then typing adjacent[12][12].remove(spring_to_break) will remove (11,11) from the list of adjacent particles. You will need to do the same thing for the particle at (11,11) removing (12,12). Since these are lists of tuples, if you save each coordinate separately then you need to add extra brackets in the remove statement. <br>
After iterating over the lattice array, the fracturing function should have selected a spring to break. Do not delete springs between particles and boundaries. If the selected neighbour is a boundary then it has no list of neighbours in the adjacent array and the entry is equal to 0.

In [ ]:
#Task 2
for h in range(50):
    mbreak = 0
    spring_to_break = 0
    spring_x, spring_y = 0, 0
    # using var_i = var[h][i][j] while iterating over the array will give you a new value of var_i
    
    
    
    # make a plot every 5 steps
    if (h%5==0)
        x = lattice[:, :, 0]
        x = x[x!=0]
        y = lattice[:, :, 1]
        y = y[y!=0]

        plt.axis('off')
        plt.rcParams["figure.figsize"] = (10,13)
        plt.scatter(x, y, s = 15)
        plt.show()

In [ ]:
t_lattice = np.loadtxt("tests/testsolve10.txt")
if (np.array_equal(lattice.reshape(100*100*2*1), t_lattice)):
    print("Passed testcase 10")
else:
    print("Failed testcase 10")

In [ ]:
#data frame for stress plots

diff_stress = np.zeros([n,n])
mean_stress = np.zeros([n,n])
stress_calculation()

x = lattice[2:n-2, 2:n-2, 0].flatten()
y = lattice[2:n-2, 2:n-2, 1].flatten()
diff_stress = diff_stress[2:n-2, 2:n-2].flatten()
diff_stress = diff_stress[x!=0]

mean_stress = mean_stress[2:n-2, 2:n-2].flatten()
mean_stress = mean_stress[x!=0]

df = pd.DataFrame({'x': x[x!=0],
                   'y': y[y!=0],
                   'Differential stress': diff_stress,
                   'Mean stress': mean_stress})

In [ ]:
df.plot.scatter('x', 'y', s=15, c='Differential stress', cmap='RdPu')
plt.axis('off')
plt.show()

In [ ]:
df.plot.scatter('x', 'y', s=15, c='Mean stress', cmap='YlOrRd')
plt.axis('off')
plt.show()

### Extra example
A high number of steps can take longer to run. You can use the data set below to explore fractures created over 2000 steps. Remove the '#' to load the lattice with the corresponding adjacency array and plot it. Do a stress calculation, then create a new data frame or run the 3 cells above to make the stress plots. You can also experiment with changing the equilibrium length and force calculation.

In [ ]:
#lattice = np.loadtxt('tests/lattice2000steps.txt').reshape(100, 100, 2, 1)
#with open("adj.txt", "r") as fp:
#   adjacent = json.load(fp)

In [ ]:
x = lattice[:, :, 0]
x = x[x!=0]
y = lattice[:, :, 1]
y = y[y!=0]

plt.axis('off')
plt.rcParams["figure.figsize"] = (10,13)
plt.scatter(x, y, s = 15)
plt.show()